Ideia: 
- Se tiver tempo, podemos fazer um streamlit separando em 3 áreas de atuação: Análise de dados, Ciência de dados e Engenharia de Dados. Assim poderia colocar essas três opções em um sidebar e fazer a mesma análise dentro para cada uma delas. Acho que seria algo bem interessante se tivermos tempo
- Por enquanto, fazemos tudo para análise de dados nesse notebook tentando deixar as coisas parametrizadas.

# Configuração inicial

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import plotly as px
import re

In [2]:
pd.set_option('display.max_columns', 500)

# Funções

In [92]:
def get_columns_part(df,parts = []):
    colunas = []
    for column in df.columns:
        column_formmatado = column[0]
        if column_formmatado[:2] in parts:
            colunas.append(column)
    return colunas

def clean_column_names(columns_df):
    try:
        new_names = [col[1] for col in columns_df] 
    except:
        new_names = columns_df
    
    return new_names

def get_columns_question(df,part,question):
    df_part = df[get_columns_part(df,parts = [part])]
    columns = [col for col in df_part.columns if col[0].split('_')[1]== question]
    return columns

def get_question(df,part,question):
    df_question = df[get_columns_question(df_career,part,question)]
    
    df_question.columns = clean_column_names(df_question.columns)
    
    level_0_name = df_question.columns[0]
    
    df_question.columns = pd.MultiIndex.from_product([[level_0_name],df_question.columns])
    df_question=df_question[df_question.columns[1:]]
    
    return df_question

def range_char(start, stop):
    
    return (chr(n) for n in range(ord(start), ord(stop) + 1))

def get_part(df,part):
    cols_part = get_columns_part(df,parts=[part])
    
    min_question = cols_part[1][0].split('_')[1]
    max_question = cols_part[-1][0].split('_')[1]
    
    questions = range_char(min_question,max_question)
    
    df_career_part = pd.concat([get_question(df_career,part,question) for question in questions],axis=1)
    return df_career_part

Configurações gerais do dataframe principal

In [112]:
df_all = pd.read_csv('./datasets/State of Data 2021 - Dataset - Pgina1.csv')
df_all.columns=[col.replace("'s","s") for col in df_all.columns]
df_all.columns = [literal_eval(col) for col in df_all.columns]
df_all.columns = [(col[0].strip(),col[1].strip()) for col in df_all.columns]
df_all.head()

/tmp/ipykernel_14892/2497512300.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv('./datasets/State of Data 2021 - Dataset - Pgina1.csv')


,"(P0, id)","(P1_a, Idade)","(P1_a_a, Faixa idade)","(P1_b, Genero)","(P1_e, Estado onde mora)","(P1_e_a, uf onde mora)","(P1_e_b, Regiao onde mora)","(P1_g_b, Regiao de origem)","(P1_g_c, Mudou de Estado?)","(P1_h, Nivel de Ensino)","(P1_i, Área de Formação)","(P2_a, Qual sua situação atual de trabalho?)","(P2_b, Setor)","(P2_c, Numero de Funcionarios)","(P2_d, Gestor?)","(P2_e, Cargo como Gestor)","(P2_f, Cargo Atual)","(P2_g, Nivel)","(P2_h, Faixa salarial)","(P2_i, Quanto tempo de experiência na área de dados você tem?)","(P2_j, Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?)","(P2_k, Você está satisfeito na sua empresa atual?)","(P2_l, Qual o principal motivo da sua insatisfação com a empresa atual?)","(P2_l_a, Falta de oportunidade de crescimento no emprego atual)","(P2_l_b, Salário atual não corresponde ao mercado)","(P2_l_c, Não tenho uma boa relação com meu líder/gestor)","(P2_l_d, Gostaria de trabalhar em em outra área de atuação)","(P2_l_e, Gostaria de receber mais benefícios)","(P2_l_f, O clima de trabalho/ambiente não é bom)","(P2_l_g, Falta de maturidade analítica na empresa)","(P2_m, Você participou de entrevistas de emprego nos últimos 6 meses?)","(P2_n, Você pretende mudar de emprego nos próximos 6 meses?)","(P2_o, Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?)","(P2_o_a, Remuneração/Salário)","(P2_o_b, Benefícios)","(P2_o_c, Propósito do trabalho e da empresa)","(P2_o_d, Flexibilidade de trabalho remoto)","(P2_o_e, Ambiente e clima de trabalho)","(P2_o_f, Oportunidade de aprendizado e trabalhar com referências na área)","(P2_o_g, Plano de carreira e oportunidades de crescimento profissional)","(P2_o_h, Maturidade da empresa em termos de tecnologia e dados)","(P2_o_i, Qualidade dos gestores e líderes)","(P2_o_j, Reputação que a empresa tem no mercado)","(P2_q, Atualmente qual a sua forma de trabalho?)","(P2_r, Qual a forma de trabalho ideal para você?)","(P2_s, Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?)","(P3_a, Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?)","(P3_b, Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?)","(P3_b_a, Analytics Engineer)","(P3_b_b, Engenharia de Dados/Data Engineer)","(P3_b_c, Analista de Dados/Data Analyst)","(P3_b_d, Cientista de Dados/Data Scientist)","(P3_b_e, Database Administrator/DBA)","(P3_b_f, Analista de Business Intelligence/BI)","(P3_b_g, Arquiteto de Dados/Data Architect)","(P3_b_h, Data Product Manager/DPM)","(P3_b_i, Business Analyst)","(P3_c, Quais dessas responsabilidades fazem parte da sua rotina atual de trabalho como gestor?)","(P3_c_a, Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.)","(P3_c_b, Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.)","(P3_c_c, Atração, seleção e contratação de talentos para o time de dados.)","(P3_c_d, Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.)","(P3_c_e, Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.)","(P3_c_f, Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.)","(P3_c_g, Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.)","(P3_c_h, Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.)","(P3_c_i, Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.)","(P3_c_j, Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.)","(P3_c_k, Gestão de pessoas, apoio no desenvolvimen

In [5]:
df_all.shape

(2645, 356)

# Base de dados e contexto

A base de dados foi obtida a partir do [link](https://www.kaggle.com/datasets/datahackers/state-of-data-2021 'dataset').
Ela consiste numa pesquisa feita para entender o mecado de trabalho na área de dados do Brasil e é dividido em sete partes:

- Parte 1 - Dados demográficos
- Parte 2 - Dados sobre carreira
- Parte 3 - Desafios dos gestores de times de dados
- Parte 4 - Conhecimentos na área de dados
- Parte 5 - Objetivos na área de dados
- Parte 6 - Conhecimentos em Engenharia de Dados/DE
- Parte 7 - Conhecimentos em Análise de Dados/DA
- Parte 8 - Conhecimentos em Ciências de Dados/DS
- Parte 9 - Sobre a comunidade Data Hackers

O objetivos será explorar algumas informações das Partes 1, 2, 4 e 7 e responder as seguintes perguntas:
- <mark style="background-color: #FFFF00">Pergunta 1?</mark>  
- <mark style="background-color: #FFFF00">Pergunta 1?</mark>
- <mark style="background-color: #FFFF00">...</mark>
- <mark style="background-color: #FFFF00">Pergunta n?</mark>

## Lendo dataset para carreira específica

In [6]:
career = 'Análise de Dados'

In [7]:
df_career = df_all[df_all[('P4_a', 'Atuacao')]==career]
df_career.head()

,"(P0, id)","(P1_a, Idade)","(P1_a_a, Faixa idade)","(P1_b, Genero)","(P1_e, Estado onde mora)","(P1_e_a, uf onde mora)","(P1_e_b, Regiao onde mora)","(P1_g_b, Regiao de origem)","(P1_g_c, Mudou de Estado?)","(P1_h, Nivel de Ensino)","(P1_i, Área de Formação)","(P2_a, Qual sua situação atual de trabalho?)","(P2_b, Setor)","(P2_c, Numero de Funcionarios)","(P2_d, Gestor?)","(P2_e, Cargo como Gestor)","(P2_f, Cargo Atual)","(P2_g, Nivel)","(P2_h, Faixa salarial)","(P2_i, Quanto tempo de experiência na área de dados você tem?)","(P2_j, Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?)","(P2_k, Você está satisfeito na sua empresa atual?)","(P2_l, Qual o principal motivo da sua insatisfação com a empresa atual?)","(P2_l_a, Falta de oportunidade de crescimento no emprego atual)","(P2_l_b, Salário atual não corresponde ao mercado)","(P2_l_c, Não tenho uma boa relação com meu líder/gestor)","(P2_l_d, Gostaria de trabalhar em em outra área de atuação)","(P2_l_e, Gostaria de receber mais benefícios)","(P2_l_f, O clima de trabalho/ambiente não é bom)","(P2_l_g, Falta de maturidade analítica na empresa)","(P2_m, Você participou de entrevistas de emprego nos últimos 6 meses?)","(P2_n, Você pretende mudar de emprego nos próximos 6 meses?)","(P2_o, Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?)","(P2_o_a, Remuneração/Salário)","(P2_o_b, Benefícios)","(P2_o_c, Propósito do trabalho e da empresa)","(P2_o_d, Flexibilidade de trabalho remoto)","(P2_o_e, Ambiente e clima de trabalho)","(P2_o_f, Oportunidade de aprendizado e trabalhar com referências na área)","(P2_o_g, Plano de carreira e oportunidades de crescimento profissional)","(P2_o_h, Maturidade da empresa em termos de tecnologia e dados)","(P2_o_i, Qualidade dos gestores e líderes)","(P2_o_j, Reputação que a empresa tem no mercado)","(P2_q, Atualmente qual a sua forma de trabalho?)","(P2_r, Qual a forma de trabalho ideal para você?)","(P2_s, Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?)","(P3_a, Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?)","(P3_b, Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?)","(P3_b_a, Analytics Engineer)","(P3_b_b, Engenharia de Dados/Data Engineer)","(P3_b_c, Analista de Dados/Data Analyst)","(P3_b_d, Cientista de Dados/Data Scientist)","(P3_b_e, Database Administrator/DBA)","(P3_b_f, Analista de Business Intelligence/BI)","(P3_b_g, Arquiteto de Dados/Data Architect)","(P3_b_h, Data Product Manager/DPM)","(P3_b_i, Business Analyst)","(P3_c, Quais dessas responsabilidades fazem parte da sua rotina atual de trabalho como gestor?)","(P3_c_a, Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.)","(P3_c_b, Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.)","(P3_c_c, Atração, seleção e contratação de talentos para o time de dados.)","(P3_c_d, Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.)","(P3_c_e, Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.)","(P3_c_f, Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.)","(P3_c_g, Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.)","(P3_c_h, Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.)","(P3_c_i, Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.)","(P3_c_j, Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.)","(P3_c_k, Gestão de pessoas, apoio no desenvolvimen

## Lendo a Parte 1

A P1 consiste em dados gerais das pessoas que responderam a pesquisa.

### Ideias de perguntas
- Relacionar nível de ensino com salário
    - Normalmente, dizem que ter um mestrado é importante para ciênciad e dados. 
    - Existe uma relação com pós-graduação e atuação em dados?
- Analisar a frequência de área de formação. Colocar em grupos maiores Exatas, Tecnologicas, Humanas, Natureza?
- Idade média dos profissionais
- Um profissional de dados precisa estar disposto a se mudar para trabalhar?

In [8]:
df_dem = df_career[get_columns_part(df_all,['P1'])]
df_dem.head()

,"(P1_a, Idade)","(P1_a_a, Faixa idade)","(P1_b, Genero)","(P1_e, Estado onde mora)","(P1_e_a, uf onde mora)","(P1_e_b, Regiao onde mora)","(P1_g_b, Regiao de origem)","(P1_g_c, Mudou de Estado?)","(P1_h, Nivel de Ensino)","(P1_i, Área de Formação)"
485,35.0,35-39,Feminino,Minas Gerais (MG),MG,Sudeste,NaN,0,Pós-graduação,Computação / Engenharia de Software / Sistemas...
487,30.0,30-34,Feminino,Paraná (PR),PR,Sul,NaN,0,Doutorado ou Phd,Outras Engenharias
491,26.0,25-29,Masculino,São Paulo (SP),SP,Sudeste,Sul,1,Graduação/Bacharelado,Outras Engenharias
494,37.0,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,Economia/ Administração / Contabilidade / Fina...
496,18.0,17-21,Masculino,Minas Gerais (MG),MG,Sudeste,NaN,0,Não tenho graduação formal,NaN


### Alterando nome das colunas

In [9]:
df_dem.columns

Index([             ('P1_a', 'Idade'),      ('P1_a_a', 'Faixa idade'),
                   ('P1_b', 'Genero'),   ('P1_e', 'Estado onde mora'),
           ('P1_e_a', 'uf onde mora'), ('P1_e_b', 'Regiao onde mora'),
       ('P1_g_b', 'Regiao de origem'), ('P1_g_c', 'Mudou de Estado?'),
          ('P1_h', 'Nivel de Ensino'),   ('P1_i', 'Área de Formação')],
      dtype='object')

In [10]:
df_dem.columns = clean_column_names(df_dem.columns)

In [11]:
df_dem.columns

Index(['Idade', 'Faixa idade', 'Genero', 'Estado onde mora', 'uf onde mora',
       'Regiao onde mora', 'Regiao de origem', 'Mudou de Estado?',
       'Nivel de Ensino', 'Área de Formação'],
      dtype='object')

In [12]:
df_dem.head()

,Idade,Faixa idade,Genero,Estado onde mora,uf onde mora,Regiao onde mora,Regiao de origem,Mudou de Estado?,Nivel de Ensino,Área de Formação
485,35.0,35-39,Feminino,Minas Gerais (MG),MG,Sudeste,NaN,0,Pós-graduação,Computação / Engenharia de Software / Sistemas...
487,30.0,30-34,Feminino,Paraná (PR),PR,Sul,NaN,0,Doutorado ou Phd,Outras Engenharias
491,26.0,25-29,Masculino,São Paulo (SP),SP,Sudeste,Sul,1,Graduação/Bacharelado,Outras Engenharias
494,37.0,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,Economia/ Administração / Contabilidade / Fina...
496,18.0,17-21,Masculino,Minas Gerais (MG),MG,Sudeste,NaN,0,Não tenho graduação formal,NaN


Porcentagem de nulos por coluna

In [13]:
df_dem.isna().sum()/df_dem.shape[0]*100

Idade                0.942285
Faixa idade          0.000000
Genero               0.000000
Estado onde mora     1.413428
uf onde mora         1.413428
Regiao onde mora     0.000000
Regiao de origem    84.687868
Mudou de Estado?     0.000000
Nivel de Ensino      0.000000
Área de Formação     1.295642
dtype: float64

## Lendo a Parte 4

Consiste em informações sobre atuação e ferramentas/tecnologias utilizadas pelos profissionais na área de dados:

É um total de 9 perguntas em 5 temas:
- Atuação
    - Área
- Fontes de dados:
    - Fontes de dados já analisadas ou processadas
    - Fonte de dados mais utilizada
- Linguagens de programação
    - Linguagens de programação utilizadas no trabalho
    - Linguagem de programação mais utilizada no trabalho
- Bancos de dados
    - Bancos de dados utilizados no trabalho
- Cloud
    - Opções de Cloud utilizadas no trabalho
- BI
    - Ferramentas de BI utilizadas no trabalho

Ideia
- Analisar quantidade de pessoas buscando empregos, gestores e etc
- Analisar linguagens de programação mais usadas
- Analisar linguagens que mais agregam valor salarial
- Analisar clouds e sua relação com salário
- Analisar ferramentas de BI e sua relação com salário
- Analisar linguagens de bancos de dados mais comuns (P4_f)
- Existe uma discrepância de tecnologias entre aqueles que estão estudando e aquele que estão empregados? Os estudantes tem trilhado o caminho certo?
- Fazer análises considerando diferentes áreas de atuação

In [94]:
part = 'P4'
df_career_tools = get_part(df_career,part)
df_career_tools.head()

Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?  \
                                                                                                                              Atuacao   
485                                   Análise de Dados                                                                                  
487                                   Análise de Dados                                                                                  
491                                   Análise de Dados                                                                                  
494                                   Análise de Dados                                                                                  
496                                   Análise de Dados                                                                                  

    Quais das fontes de dados listadas você já analisou ou processou no trabalho?  \
                                   Dados relacionais (estruturados em bancos SQL)   
485                                                1.0                              
487                                                1.0                              
491                                                0.0                              
494                                                1.0                              
496                                                1.0                              

                                                                               \
    Dados armazenados em bancos NoSQL Imagens Textos/Documentos Vídeos Áudios   
485                               0.0     0.0               0.0    0.0    0.0   
487                               1.0     0.0               1.0    0.0    0.0   
491                               0.0     0.0               0.0    0.0    0.0   
494                               0.0     1.0               1.0    0.0    0.0   
496                               0.0     0.0               0.0    0.0    0.0   

                                      \
    Planilhas Dados georeferenciados   
485       0.0                    0.0   
487       1.0                    0.0   
491       1.0                    0.0   
494       1.0                    1.0   
496       1.0                    0.0   

    Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?  \
                                    Dados relacionais (estruturados em bancos SQL)   
485                                                1.0                               
487                                                1.0                               
491                                                0.0                               
494                                                1.0                               
496                                                1.0                               

                                                                               \
    Dados armazenados em bancos NoSQL Imagens Textos/Documentos Vídeos Áudios   
485                               0.0     0.0               0.0    0.0    0.0   
487                               0.0     0.0               0.0    0.0    0.0   
491                               0.0     0.0               0.0    0.0    0.0   
494                               0.0     0.0               0.0    0.0    0.0   
496                               0.0     0.0               0.0    0.0    0.0   

                                      \
    Planilhas Dados georeferenciados   
485       1.0                    0.0   
487       1.0                    0.0   
491       1.0                    0.0   
494       1.0                    0.0   
496       1.0                    0.0   

    Quais das linguagens listadas abaixo você utiliza no trabalho?       \
                                                               SQL    R 

In [95]:
df_career_tools = df_career_tools[df_career_tools.columns[1:]]
df_career_tools.head()

Quais das fontes de dados listadas você já analisou ou processou no trabalho?  \
                                   Dados relacionais (estruturados em bancos SQL)   
485                                                1.0                              
487                                                1.0                              
491                                                0.0                              
494                                                1.0                              
496                                                1.0                              

                                                                               \
    Dados armazenados em bancos NoSQL Imagens Textos/Documentos Vídeos Áudios   
485                               0.0     0.0               0.0    0.0    0.0   
487                               1.0     0.0               1.0    0.0    0.0   
491                               0.0     0.0               0.0    0.0    0.0   
494                               0.0     1.0               1.0    0.0    0.0   
496                               0.0     0.0               0.0    0.0    0.0   

                                      \
    Planilhas Dados georeferenciados   
485       0.0                    0.0   
487       1.0                    0.0   
491       1.0                    0.0   
494       1.0                    1.0   
496       1.0                    0.0   

    Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?  \
                                    Dados relacionais (estruturados em bancos SQL)   
485                                                1.0                               
487                                                1.0                               
491                                                0.0                               
494                                                1.0                               
496                                                1.0                               

                                                                               \
    Dados armazenados em bancos NoSQL Imagens Textos/Documentos Vídeos Áudios   
485                               0.0     0.0               0.0    0.0    0.0   
487                               0.0     0.0               0.0    0.0    0.0   
491                               0.0     0.0               0.0    0.0    0.0   
494                               0.0     0.0               0.0    0.0    0.0   
496                               0.0     0.0               0.0    0.0    0.0   

                                      \
    Planilhas Dados georeferenciados   
485       1.0                    0.0   
487       1.0                    0.0   
491       1.0                    0.0   
494       1.0                    0.0   
496       1.0                    0.0   

    Quais das linguagens listadas abaixo você utiliza no trabalho?       \
                                                               SQL    R   
485                                                1.0              0.0   
487                                                1.0              1.0   
491                                                0.0              0.0   
494                                                1.0              1.0   
496                                                1.0              0.0   

                                                                             \
    Python C/C++/C# .NET Java Julia SAS/Stata Visual Basic/VBA Scala Matlab   
485    1.0      0.0  0.0  0.0   0.0       0.0              0.0   0.0    0.0   
487    1.0      0.0  0.0  0.0   0.0       0.0              0.0   0.0    0.0   
491    0.0      0.0  0.0  0.0   0.0       0.0              0.0   0.0    0.0   
494    1.0      0.0  0.0  1.0   0.0       0.0              0.0   0.0    0.0   
496    1.0      0.0  0.0  0.0   0.0       0.0              0.0   0.0    0.0   

                           

In [96]:
df_career_tools.shape

(849, 108)

In [99]:
df_career_tools[df_career_tools.isna().any(axis=1)]

Empty DataFrame
Columns: [(Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Dados relacionais (estruturados em bancos SQL)), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Dados armazenados em bancos NoSQL), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Imagens), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Textos/Documentos), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Vídeos), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Áudios), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Planilhas), (Quais das fontes de dados listadas você já analisou ou processou no trabalho?, Dados georeferenciados), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Dados relacionais (estruturados em bancos SQL)), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Dados armazenados em bancos NoSQL), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Imagens), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Textos/Documentos), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Vídeos), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Áudios), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Planilhas), (Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?, Dados georeferenciados), (Quais das linguagens listadas abaixo você utiliza no trabalho?, SQL), (Quais das linguagens listadas abaixo você utiliza no trabalho?, R), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Python), (Quais das linguagens listadas abaixo você utiliza no trabalho?, C/C++/C#), (Quais das linguagens listadas abaixo você utiliza no trabalho?, .NET), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Java), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Julia), (Quais das linguagens listadas abaixo você utiliza no trabalho?, SAS/Stata), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Visual Basic/VBA), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Scala), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Matlab), (Quais das linguagens listadas abaixo você utiliza no trabalho?, PHP), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Javascript), (Quais das linguagens listadas abaixo você utiliza no trabalho?, Não utilizo nenhuma linguagem), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, SQL), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, R), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Python), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, C/C++/C#), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, .NET), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Java), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Julia), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, SAS/Stata), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Visual Basic/VBA), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Scala), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Matlab), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, PHP), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Javascript), (Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?, Não 

In [100]:
#dropping lines with nan
df_career_tools.dropna(axis=0,inplace=True)
df_career_tools.shape

(846, 108)

## Lendo a parte 7

In [108]:
part = 'P7'
df_career_skills = get_part(df_career,part)
df_career_skills.head()

Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?  \
            Processo e analiso dados utilizando linguagens de programação como Python, R etc.   
485                                                0.0                                          
487                                                1.0                                          
491                                                0.0                                          
494                                                0.0                                          
496                                                1.0                                          

                                                                                                     \
    Realizo construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik etc.   
485                                                1.0                                                
487                                                1.0                                                
491                                                1.0                                                
494                                                1.0                                                
496                                                1.0                                                

                                                                     \
    Utilizo APIs para extrair dados e complementar minhas análises.   
485                                                0.0                
487                                                0.0                
491                                                0.0                
494                                                1.0                
496                                                1.0                

                                                                                                                           \
    Realizo experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc.   
485                                                0.0                                                                      
487                                                0.0                                                                      
491                                                0.0                                                                      
494                                                0.0                                                                      
496                                                1.0                                                                      

                                                                                                                \
    Desenvolvo/cuido da manutenção de ETLs utilizando tecnologias como Talend, Pentaho, Airflow, Dataflow etc.   
485                                                0.0                                                           
487                                                0.0                                                           
491                                                0.0                                                           
494                                                0.0                                                           
496                                                1.0                                                           

                                                                                                               \
    Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados, Data Warehouses, Data Marts etc.   
485                                                0.0                                                          
487                                                0.0                                    

In [109]:
df_career_skills.shape

(849, 39)

In [110]:
df_career_skills[df_career_skills.isna().any(axis=1)]

Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?  \
            Processo e analiso dados utilizando linguagens de programação como Python, R etc.   
802                                                1.0                                          

                                                                                                     \
    Realizo construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik etc.   
802                                                0.0                                                

                                                                     \
    Utilizo APIs para extrair dados e complementar minhas análises.   
802                                                0.0                

                                                                                                                           \
    Realizo experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc.   
802                                                1.0                                                                      

                                                                                                                \
    Desenvolvo/cuido da manutenção de ETLs utilizando tecnologias como Talend, Pentaho, Airflow, Dataflow etc.   
802                                                0.0                                                           

                                                                                                               \
    Atuo na modelagem dos dados, com o objetivo de criar conjuntos de dados, Data Warehouses, Data Marts etc.   
802                                                0.0                                                          

                                                                                   \
    Desenvolvo/cuido da manutenção de planilhas para atender as áreas de negócio.   
802                                                0.0                              

                                                           \
    Utilizo ferramentas avançadas de estatística como SAS   
802                                                0.0      

    Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?  \
                                                                             Scripts Python   
802                                                NaN                                        

                                                                         \
    SQL & Stored Procedures Apache Airflow Luigi AWS Glue Talend Stitch   
802                     NaN            NaN   NaN      NaN    NaN    NaN   

                                                                              \
    Fivetran Google Dataflow Oracle Data Integrator IBM DataStage SAP BW ETL   
802      NaN             NaN                    NaN           NaN        NaN   

                                                                            \
    SQL Server Integration Services (SSIS) SAS Data Integration Qlik Sense   
802                                    NaN                  NaN        NaN   

                                          \
    Knime Não utilizo ferramentas de ETL   
802   NaN                            NaN   

    Sua empresa utiliza alguma das ferramentas listadas para dar mais autonomia em análise de dados para as áreas de negócio?  \
                                                                    Ferramentas de AutoML como H2O.ai, Data Robot, BigML etc.   
802                                                0.0                                                                          

                                                                        \
    ""Point and Click"" Analytics como Alteryx, Knime, Rapidminer etc.   
802                             

In [111]:
#dropping lines with nan
df_career_skills.dropna(axis=0,inplace=True)
df_career_skills.shape

(848, 39)